In [12]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2 accelerate~=1.1.1
# for facebook/mask2former-swin-small-coco-instance
!pip install -q pillow~=11.0.0

In [13]:
import torch

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

# Hugging Face 初體驗

[Hugging Face](https://huggingface.co/) 是一個專注於機器學習和自然語言處理的開源平台。它提供了大量的模型、數據集和工具，支持多種任務如文本生成、圖像分類和語音識別。用戶可以輕鬆地搜索、下載和使用這些資源來開發和部署AI應用。這個平台對研究人員和開發者來說非常有用，因為它促進了技術共享和協作。

平台提供各類開源模型及資料集，也針對不同型態的[任務](https://huggingface.co/tasks)進行分類，便於查找使用．

![Hugging Face](https://raw.githubusercontent.com/jonascheng/learning-transformer-with-huggingface/refs/heads/main/assets/images/huggingface.png)

我們來體驗幾個簡單的任務

## [Token Classification](https://huggingface.co/tasks/token-classification)

Token classification 是自然語言處理中的一種任務，旨在對文本中的每個 token（通常是單詞或子詞）進行分類。這種技術可以用來解決多種問題，以下是幾個例子：

* 命名實體識別（NER）：識別文本中的人名、地名、組織名等實體。例如，從句子「John is working in New York」中識別出「John」是人名，「New York」是地名。
* 詞性標註（POS tagging）：標註每個單詞的詞性，如名詞、動詞、形容詞等。例如，從句子「The quick brown fox jumps over the lazy dog」中標註出「quick」是形容詞，「jumps」是動詞。


In [ ]:
from transformers import pipeline

# 讓 pipeline 自動選擇模型
ner_classifier = pipeline(
  task="token-classification",
  device=device,)
ner_classifier("John is working in New York")

從回傳的結構化資料，我們可以看到每個 token 的 `word` 和 `entity`，這裡的 `entity` 是指該 token 的分類結果。

```json
[
  {
    'entity': 'I-PER',
    'word': 'John'
  }, {
    'entity': 'I-LOC',
    'word': 'New'
  }, {
    'entity': 'I-LOC',
    'word': 'York'
  }
]
```

In [ ]:
# 詞性標註（POS tagging）, 指定模型
model_name = "vblagoje/bert-english-uncased-finetuned-pos"
pos_classifier = pipeline(
  task="token-classification",
  model=model_name,
  device=device,)
pos_classifier("The quick brown fox jumps over the lazy dog")

```json
[
  {
    'entity': 'DET', # 限定詞
    'word': 'the'
  }, {
    'entity': 'ADJ', # 形容詞
    'word': 'quick'
  }, {
    'entity': 'ADJ',
    'word': 'brown'
  }, {
    'entity': 'NOUN', # 名詞
    'word': 'fox'
  }, {
    'entity': 'VERB', # 動詞
    'word': 'jumps'
  }, {
    'entity': 'ADP', # 介系詞
    'word': 'over'
  },
  ...
]
```

## [Zero-Shot Classification](https://huggingface.co/tasks/zero-shot-classification)

Zero-Shot Classification 旨在不需要特定訓練數據的情況下，對文本進行分類。這意味著模型可以在沒有見過特定類別的訓練樣本的情況下，直接對新類別進行分類。這種技術依賴於預訓練模型的強大語言理解能力。以下是幾個可以透過 Zero-Shot Classification 解決的例子：

* 情感分析：在沒有特定情感標註數據的情況下，對文本進行正面、負面或中性情感分類。例如，對一條新的社交媒體評論進行情感分類。
* 主題分類：對新聞文章進行主題分類，如政治、經濟、科技等，而不需要針對每個主題進行單獨訓練。例如，將一篇新的新聞文章分類為「科技」。

Zero-Shot Classification 的優勢在於其靈活性和廣泛應用，特別是在缺乏標註數據的情況下。


In [ ]:
from transformers import pipeline

model_name = "facebook/bart-large-mnli"
pipe = pipeline(
  task="zero-shot-classification",
  model=model_name,
  device=device,)
pipe("I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)

從回傳的結構化資料，我們可以知道這是一個關於 `phone` 且 `urgent` 的用戶反饋。

## Image Classification

Transformer 架構不僅適用於自然語言處理，也可以應用於圖像處理。這裡我們使用一個預訓練的圖像分類模型來識別圖像中的物體。

![貓](https://live.staticflickr.com/5670/30942263876_969b93e461_b.jpg)

來源: https://live.staticflickr.com/5670/30942263876_969b93e461_b.jpg

In [ ]:
from transformers import pipeline

model_name = "google/vit-base-patch16-224"
pipe = pipeline(
  task="image-classification",
  model=model_name,
  device=device,)
pipe("https://live.staticflickr.com/5670/30942263876_969b93e461_b.jpg")


如果對於 Vision Transformer (ViT) 與 CNN 感興趣，可以參考 [Vision Transformer vs. CNN: A Comparison of Two Image Processing Giants](https://medium.com/@hassaanidrees7/vision-transformer-vs-cnn-a-comparison-of-two-image-processing-giants-d6c85296f34f) 這篇文章。課程將專注在文本處理，所以這裡只是簡單展示圖像分類的例子。

# 文本生成策略

文本生成對於許多自然語言處理（NLP）任務至關重要，例如開放式文本生成 (open-ended text generation)、摘要、翻譯等。它還在許多以文本為輸出的混合模態 (mixed-modality) 應用中發揮作用，如語音轉文本和圖像轉文本。一些常見可以生成文本的模型包括 GPT2、OpenAI GPT、Bart、T5 等。

請注意，`generate()` 方法的輸入取決於模型的模態。這些輸入由模型的預處理類（如 `AutoTokenizer` 或 `AutoProcessor`）返回。如果模型的預處理器創建了多種類型的輸入，請將所有輸入傳遞給 `generate()`。

選擇生成文本的輸出標記的過程稱為解碼 (decoding)，您可以自定義 `generate()` 方法使用的解碼策略。修改解碼策略不會改變任何可訓練參數的值。然而，它可以顯著影響生成輸出的質量。

## 預設文本生成策略

模型的解碼 (decoding) 策略在其生成配置中定義。當在 `pipeline()` 中使用預訓練模型進行推理時，這些模型會調用 `PreTrainedModel.generate()` 方法，該方法在內部使用默認的生成配置。如果沒有與模型一起保存自定義配置，則使用默認配置。

In [ ]:
from transformers import pipeline

model_name = "distilbert/distilgpt2"

pipe = pipeline(
  task="text-generation",
  model=model_name,
  device=device,)

pipe("Once upon a time")[0]["generated_text"]

當您顯式加載模型時，可以通過 `model.generation_config` 檢查隨模型附帶的生成配置：

In [19]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
model.generation_config

`model.generation_config` 只會顯示與默認生成配置不同的值，不會列出任何默認值。

默認生成配置為

* 輸出與輸入提示結合的大小限制為最多 20 個標記，以避免資源限制。

* 默認的解碼策略是貪婪搜索 (greedy search)，這是最簡單的解碼策略，選擇概率最高的標記作為下一個標記。對於許多任務和小量輸出，這種方法效果很好。然而，當用於生成較長的輸出時，貪婪搜索可能會開始產生高度重複的結果。

預處理輸入時，`AutoTokenizer` 會自動檢測模型的模態 (modality)，並返回適當的輸入。這些輸入可以是 `input_ids`、`attention_mask`、`decoder_input_ids`、`decoder_attention_mask` 等。

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer("Once upon a time", return_tensors="pt")

In [ ]:
from pprint import pprint

pprint(inputs)

這些輸入可以直接傳遞給 `generate()` 方法，生成文本的輸出將基於模型的模態和生成策略。

In [ ]:
outputs = model.generate(**inputs)

In [ ]:
pprint(outputs)

輸出的文本是編碼後的標記，您可以使用模型的 `tokenizer` 將其解碼為人類可讀的文本。

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

## 客製化文本生成策略

您可以通過將參數及其值直接傳遞給 `generate()` 方法來覆蓋任何 `generation_config`：

* `num_beams`：表示使用束搜索（beam search）策略進行生成，並且束的數量為 4。束搜索是一種解碼策略，它在每一步選擇多個（這裡是 4 個）最有希望的候選標記，並在最終選擇最佳序列。這可以提高生成文本的質量和多樣性。

* `do_sample`：表示在生成過程中啟用隨機抽樣。這意味著在每一步生成時，模型會根據概率分佈隨機選擇下一個標記，而不是總是選擇概率最高的標記。這可以增加生成文本的多樣性，減少重複和模式化的結果。

In [ ]:
outputs = model.generate(**inputs, num_beams=4, do_sample=True)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)